In [1]:
%load_ext autoreload
%autoreload

import pandas as pd; pd.set_option('display.max_columns', 30); pd.set_option('display.max_rows', 100)
import numpy as np
from cmlreaders import CMLReader, get_data_index
import ptsa
import sys
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sb
from pylab import *

from copy import copy
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/SWRrefactored/code/SWR_modules/')
from SWRmodule import *
from general import * #superVstack,findInd,findAinB
from power_functions import z_score, process_power, load_z_scored_power

base = '/home1/john/SWRrefactored'
sys.path.append(f'{base}/code/')

from load_data_numpy import load_data_np
from SWRmodule import triangleSmooth
# from comodulogram import remove_session_string, get_filtered_signal
# from fooof import FOOOF

import statsmodels.formula.api as smf
from scipy.signal import decimate, resample, hilbert, welch, spectrogram
from mne.time_frequency import tfr_array_morlet

In [8]:
## Params ##

# for now the plotting code only works with one region at a time
# The four possible regions I've saved the data: ['ENT'] # ['AMY'] ['HPC'] ['ACC_OF']

# [''] for all subregions
# HPC: ['ca1'],['dg'] Note these are at IRI = 2000
# FC: ['acgg'],['org'],['mfc'],['gre']. Note these are at IRI = 5000
# subregion = ['ca1'] 

region_name = 'HPC'

exp = 'catFR1' # I only provided 'catFR1' data so far

selected_period = 'countdown'
# selected_period = 'distractor'

savePath = f'{base}/figures/'

In [10]:
import glob
import pickle

path_name = f'/scratch/john/SWRrefactored/patient_info/{exp}/{selected_period}/'

# 1. Get a list of all .p files in `path_name`
pickle_files = sorted(glob.glob(os.path.join(path_name, '*.p')))

# 2. Create a list (or dict) to store the loaded data
all_data = {}

# Load the first num_to_load
num_to_load = 10

# 3. Loop through each file and load it
for pfile in pickle_files[0:num_to_load]:
    with open(pfile, 'rb') as f:
        data = pickle.load(f)
    # You can choose to store the data in different ways:
    #   Option A) Append to a list
#     all_data.append(data)
    
    #   Option B) Store in a dict keyed by filename
    fname = os.path.basename(pfile)
    all_data[fname] = data

print(f"Loaded {len(all_data)} files.")


Loaded 10 files.


In [11]:
all_data.keys()

dict_keys(['RAW_catFR1_R1004D_0_HPC_countdown.p', 'RAW_catFR1_R1015J_0_HPC_countdown.p', 'RAW_catFR1_R1024E_0_HPC_countdown.p', 'RAW_catFR1_R1031M_0_HPC_countdown.p', 'RAW_catFR1_R1032D_0_HPC_countdown.p', 'RAW_catFR1_R1032D_1_HPC_countdown.p', 'RAW_catFR1_R1035M_0_HPC_countdown.p', 'RAW_catFR1_R1061T_0_HPC_countdown.p', 'RAW_catFR1_R1065J_0_HPC_countdown.p', 'RAW_catFR1_R1065J_1_HPC_countdown.p'])

In [14]:
all_data['RAW_catFR1_R1004D_0_HPC_countdown.p'].keys()
all_data['RAW_catFR1_R1004D_0_HPC_countdown.p']

dict_keys(['elec_regions', 'elec_labels', 'raw_eeg', 'time_add_save', 'events_df', 'electrode_coords', 'eeg_buffer', 'kept_lists', 'min_duration', 'sampling_rate'])

{'elec_regions': array(['left ca1', 'left dg', 'left dg', 'left ca1'], dtype='<U23'),
 'elec_labels': 53    LOTD2-LOTD3
 54    LOTD3-LOTD4
 55    LOTD4-LOTD5
 56    LOTD5-LOTD6
 Name: label, dtype: object,
 'raw_eeg': array([[[  29.,   31.,   29., ...,   11.,   13.,   16.],
         [  10.,   13.,   15., ...,   32.,   29.,   26.],
         [ -11.,  -12.,  -11., ...,  -12.,  -11.,  -10.],
         [ -38.,  -34.,  -33., ...,   -9.,   -8.,   -9.]],
 
        [[  24.,   26.,   27., ...,  -20.,  -18.,  -17.],
         [  39.,   39.,   39., ...,   35.,   36.,   38.],
         [ -18.,  -17.,  -15., ...,    5.,    5.,    4.],
         [ -56.,  -58.,  -60., ...,  -25.,  -26.,  -27.]],
 
        [[   8.,    8.,    6., ...,  -10.,   -8.,   -8.],
         [  67.,   67.,   67., ...,  -45.,  -45.,  -44.],
         [  20.,   19.,   18., ...,   26.,   24.,   23.],
         [-108., -107., -106., ...,  -19.,  -19.,  -20.]],
 
        ...,
 
        [[  19.,   20.,   21., ...,    9.,    9.,    8.],
     